Make sure you extracted following files into the same directory as this notebook:
- training_50_dih.joblib
- validation_dih.joblib
- le.joblib
- ohe.joblib

And run 'pip install joblib'

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import glob
import tensorflow as tf
import time

import data_loader as dl
import data_transformer as dt

Modify the path below

In [2]:
# change this path to match the path of where you have the
# training_50 and validation files
txt_data_path = '/home/mikey/Data/ProteinNet/casp7_txt/'

The data_lim is important because so far the full 13000 proteins crashes my system with 16Gb of ram

Load training data

In [5]:
file_name = 'training_50'
data_lim = 8000
prim_train, evo_train, dih_train, mask_train = (dl.parse_primary_from_file(txt_data_path + file_name, data_lim), 
                                                dl.parse_evolutionary_from_file(txt_data_path + file_name, data_lim),
                                                dl.load_file('./'+file_name+'_dih.joblib'),
                                                dl.parse_mask_from_file(txt_data_path + file_name, data_lim))
dih_train = dih_train[:data_lim]
len(prim_train), prim_train[0].shape, len(evo_train), evo_train[0].shape, len(dih_train), dih_train[0].shape, len(mask_train), mask_train[0].shape

Loaded data and filtered line endings
Extracted primary data
Encoded primary sequences
Loaded data and filtered line endings
Extracted evolutionary data


100%|██████████| 8000/8000 [00:00<00:00, 74953.50it/s]


Grouped 21's together
Loaded data and filtered line endings
Extracted mask data


(8000, (70, 20), 8000, (70, 21), 13024, (70, 3), 8000, (70,))

Load validation data

In [6]:
file_name = 'validation'
prim_valid, evo_valid, dih_valid, mask_valid = (dl.parse_primary_from_file(txt_data_path + file_name), 
                                                dl.parse_evolutionary_from_file(txt_data_path + file_name),
                                                dl.load_file('./'+ file_name + '_dih.joblib'),
                                                dl.parse_mask_from_file(txt_data_path + file_name))
len(prim_valid), prim_valid[0].shape, len(evo_valid), evo_valid[0].shape, len(dih_valid), dih_valid[0].shape, len(mask_valid), mask_valid[0].shape

Loaded data and filtered line endings
Extracted primary data
Encoded primary sequences
Loaded data and filtered line endings
Extracted evolutionary data


100%|██████████| 224/224 [00:00<00:00, 54496.76it/s]

Grouped 21's together
Loaded data and filtered line endings
Extracted mask data


(224, (269, 20), 224, (269, 21), 224, (269, 3), 224, (269,))

Pad the data and limit protein length. Prepare the data for training

In [12]:
max_len = 500
prim_, evo_, dih_, mask_ = dt.limit_length_and_pad(prim_train, evo_train, dih_train, mask_train, max_len)
x_train, y_train = np.concatenate([prim_, evo_], axis=2), dih_
prim_v, evo_v, dih_v, mask_v = dt.limit_length_and_pad(prim_valid, evo_valid, dih_valid, mask_valid, max_len)
x_valid, y_valid = np.concatenate([prim_v, evo_v], axis=2), dih_v

padded
(7623,) (70, 20)
padded
(7623,) (70, 21)
padded
(7623,) (70, 3)
padded
(7623,) (70, 1)
padded
(213,) (269, 20)
padded
(213,) (269, 21)
padded
(213,) (269, 3)
padded
(213,) (269, 1)


In [13]:
x_train.shape

(7623, 500, 41)

In [14]:
x_train[np.logical_not(mask_)] = -1.
x_valid[np.logical_not(mask_v)] = -1.

In [15]:
histories = []

Run a simple regression task to check if it trains on the data

In [ ]:
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Lambda, Masking, concatenate
from keras.layers import LSTM, Conv1D, Input

inputs = Input(shape=(max_len,41))
x1 = Masking(mask_value=-1.)(inputs)
x2 = LSTM(10, return_sequences=True)(x1)
x12 = concatenate([x1,x2])
x3 = LSTM(10, return_sequences=True)(x12)
x4 = Dense(3, activation='tanh')(x3)
y = Lambda(lambda x: x*np.pi)(x4)
model = Model(inputs=inputs, outputs=y)

# model = Sequential()
# model.add(Masking(mask_value=-1.))
# model.add(LSTM(128, return_sequences=True))
# # model.add(LSTM(128, return_sequences=True))
# # model.add(Conv1D(32, 15, padding='same'))
# # model.add(Conv1D(64, 15, padding='same'))
# # model.add(Conv1D(128, 15, padding='same'))
# # model.add(Dropout(0.5))
# model.add(Dense(3, activation='tanh'))
# model.add(Lambda(lambda x: x*np.pi))

model.compile(loss='mean_absolute_error',
              optimizer='adam',
              metrics=['mean_absolute_error'])

history = model.fit(x_train, y_train, validation_data=(x_valid, y_valid), batch_size=100, epochs=10)
histories.append(history)
# score = model.evaluate(x_valid, y_valid, batch_size=16)

Using TensorFlow backend.


Train on 7623 samples, validate on 213 samples
Epoch 1/10
7623/7623 [==============================] - 82s 11ms/step - loss: 0.5948 - mean_absolute_error: 0.5948 - val_loss: 0.5815 - val_mean_absolute_error: 0.5815
Epoch 2/10
7623/7623 [==============================] - 81s 11ms/step - loss: 0.5593 - mean_absolute_error: 0.5593 - val_loss: 0.5682 - val_mean_absolute_error: 0.5682
Epoch 3/10
7623/7623 [==============================] - 81s 11ms/step - loss: 0.5508 - mean_absolute_error: 0.5508 - val_loss: 0.5614 - val_mean_absolute_error: 0.5614
Epoch 4/10
7623/7623 [==============================] - 82s 11ms/step - loss: 0.5450 - mean_absolute_error: 0.5450 - val_loss: 0.5558 - val_mean_absolute_error: 0.5558
Epoch 5/10
7623/7623 [==============================] - 82s 11ms/step - loss: 0.5401 - mean_absolute_error: 0.5401 - val_loss: 0.5513 - val_mean_absolute_error: 0.5513
Epoch 6/10
7623/7623 [==============================] - 82s 11ms/step - loss: 0.5364 - mean_absolute_error: 0.536